In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from certa import local_explain, triangles_method
from certa.eval import expl_eval

from certa.utils import merge_sources
from certa.local_explain import get_original_prediction
from models.bert import EMTERModel

In [2]:
model = EMTERModel()
model = model.load('models/bert/abt_buy')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [3]:
def predict_fn(x, model, ignore_columns=['ltable_id', 'rtable_id', 'label']):
    return model.predict(x)

In [6]:
datadir = 'datasets/abt_buy'
lsource = pd.read_csv(datadir + '/tableA.csv')
rsource = pd.read_csv(datadir + '/tableB.csv')
gt = pd.read_csv(datadir + '/train.csv')
valid = pd.read_csv(datadir + '/valid.csv')
test = pd.read_csv(datadir + '/test.csv')

In [7]:
test_df = merge_sources(test, 'ltable_', 'rtable_', lsource, rsource, ['label'], []).dropna()

In [59]:
rand_row = test_df[test_df['label']==1].iloc[4]
l_id = int(rand_row['ltable_id'])
l_tuple = lsource.iloc[l_id]
r_id = int(rand_row['rtable_id'])
r_tuple = rsource.iloc[r_id]
label = rand_row['label']
label

1

In [61]:
l_tuple

id                                                           631
name           sony 3.1 channel home theater surround system ...
description    sony 3.1 channel home theater surround system ...
price                                                        299
Name: 631, dtype: object

In [62]:
r_tuple

id                                                  864
name                  sony ht-ct100 home theater system
description    , 3.1 speakers progressive scan 250w rms
price                                            279.95
Name: 864, dtype: object

In [60]:
# false negative
prediction = get_original_prediction(l_tuple, r_tuple, model, predict_fn)
predicted_class = np.argmax(prediction)
predicted_class

0

In [63]:
tmin = 0.5
tmax = 0.5
local_samples, gleft_df, gright_df = local_explain.dataset_local(l_tuple, r_tuple, model, lsource, rsource, datadir,
                                                                            tmin, tmax, predict_fn, num_triangles=10,
                                                                            class_to_explain=predicted_class, use_predict=True,
                                                                            max_predict=500)

/home/tteofili/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/home/tteofili/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [75]:
explanation, flipped_pred, triangles = triangles_method.explainSamples(local_samples,
                                                      [pd.concat([lsource, gright_df]),
                                                       pd.concat([rsource, gleft_df])],
                                                      model, predict_fn, predicted_class, 
                                                                       attribute_combine=False,
                                                                      check=False, discard_bad=False)
explanation

/home/tteofili/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tteofili/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/home/tteofili/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tteofili/.local/lib/python3.6/site-packages/sklearn/

defaultdict(int, {'description': 0.1})

In [70]:
exp = list(explanation)[0]
e_attrs = exp.split('/')
e_score = explanation[exp]
expl_evaluation = expl_eval(predicted_class, e_attrs, e_score, lsource, l_tuple, model,
                            prediction, rsource,
                            r_tuple, predict_fn)

In [71]:
expl_evaluation

,match,e_score,drop,perturb,flip,impact,attributes,e_size,prediction,row
0,0,0.1,0.006896,drop,0,0,[description],1,0.993335,631-864
1,0,0.1,0.000648,sample,0,0,[description],1,0.993335,631-864
